In [1]:
# You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one! Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

# Pres_A : Percentage of preservative A in the mix
# Pres_B : Percentage of preservative B in the mix
# Pres_C : Percentage of preservative C in the mix
# Pres_D : Percentage of preservative D in the mix
# Spoiled: Label indicating whether or not the dog food batch was spoiled.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('treeMethods').getOrCreate()

In [3]:
data = spark.read.csv('/FileStore/tables/dog_food.csv', inferSchema=True, header=True)

In [4]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
summary| A| B| C| D| Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
 count| 490| 490| 490| 490| 490|
 mean| 5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
 stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
 min| 1| 1| 5.0| 1| 0.0|
 max| 10| 10| 14.0| 10| 1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+

In [5]:
data.head(5)

Out[ 7 ]: 
[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0),
 Row(A=5, B=6, C=12.0, D=7, Spoiled=1.0),
 Row(A=6, B=2, C=13.0, D=6, Spoiled=1.0),
 Row(A=4, B=2, C=12.0, D=1, Spoiled=1.0),
 Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)]

In [6]:
data.columns

Out[ 8 ]: ['A', 'B', 'C', 'D', 'Spoiled']

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(
  inputCols=['A', 'B', 'C', 'D'],
  outputCol='features')

In [9]:
output = assembler.transform(data)

In [10]:
final_data = output.select('features', 'Spoiled')

In [11]:
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier

In [12]:
rfc = DecisionTreeClassifier(labelCol='Spoiled', featuresCol='features')

In [13]:
rfc_model = rfc.fit(final_data)

In [14]:
rfc_model.featureImportances

Out[ 24 ]: SparseVector(4, {1: 0.0019, 2: 0.9832, 3: 0.0149})

In [15]:
rfc = RandomForestClassifier(labelCol='Spoiled', featuresCol='features')
rfc_model = rfc.fit(final_data)
rfc_model.featureImportances

Out[ 26 ]: SparseVector(4, {0: 0.0202, 1: 0.0172, 2: 0.9402, 3: 0.0224})

In [16]:
#Based on the importances of the two methods, Chemical C might be the one causing the spoilage.